In [0]:
import pandas as pd
import numpy as np
import os
import dotenv
import fastText
from tqdm import tqdm
import gc

dotenv.load_dotenv('.env')
tqdm.pandas()
types = ['train', 'test']
str_cols = ['description']

In [2]:
for key in types:
  path = './data/%s_prep.snappy.parquet'%key
  !gsutil cp gs://{os.environ['GCP_BUCKET']}/{key}_prep.snappy.parquet {path}

Copying gs://kaggle-195720-avito-demand-prediction/train_prep.snappy.parquet...
/ [1 files][465.7 MiB/465.7 MiB]                                                
Operation completed over 1 objects/465.7 MiB.                                    
Copying gs://kaggle-195720-avito-demand-prediction/test_prep.snappy.parquet...
/ [1 files][163.8 MiB/163.8 MiB]                                                
Operation completed over 1 objects/163.8 MiB.                                    


In [6]:
'abc'.replace('ab', 'c')

'cc'

In [3]:
t = 'train'
col = 'description'
ser = pd.read_parquet('./data/%s_prep.snappy.parquet'%(t), columns=[col])[col].fillna('').progress_apply(lambda x: x.replace('\n', ' '))

100%|██████████| 1503424/1503424 [00:03<00:00, 488476.34it/s]


In [4]:
ft = fastText.load_model('./model/russian_fasttext.bin')
cols = ['%s_ft_%d'%(col, i) for i in range(ft.get_dimension())]
ser = ser.progress_apply(ft.get_sentence_vector)

del ft
gc.collect()


100%|██████████| 1503424/1503424 [03:56<00:00, 6360.63it/s]


0

In [0]:
df = pd.DataFrame(np.array(ser.tolist()), columns=cols)

In [0]:
path = './data/%s_%s_fasttext.snappy.parquet'%(t, col)
df.to_parquet(path)


In [0]:
for col in str_cols:
  for t in types:
    print('Processing %s, %s'%(t, col))
    
    ser = pd.read_parquet('./data/%s_prep.snappy.parquet'%(t), columns=[col])[col].fillna('').progress_apply(lambda x: x.replace('\n', ' '))

    ft = fastText.load_model('./model/russian_fasttext.bin')
    cols = ['%s_ft_%d'%(col, i) for i in range(ft.get_dimension())]
    ser = ser.progress_apply(ft.get_sentence_vector)
    
    del ft
    gc.collect()
    
    df = pd.DataFrame(np.array(ser.tolist()), columns=cols)
    
    del ser
    gc.collect()
    
    print('Saving %s, %s'%(t, col))
    path = './data/%s_%s_fasttext.snappy.parquet'%(t, col)
    df.to_parquet(path)
    !gsutil cp {path} gs://{os.environ['GCP_BUCKET']}/{t}_{col}_prep.snappy.parquet
      
    del df
    gc.collect()

Processing train, description


 37%|███▋      | 555899/1503424 [01:26<02:18, 6852.31it/s]

100%|██████████| 1503424/1503424 [03:57<00:00, 6333.27it/s]


Saving train, description


In [0]:
import pandas as pd
train = pd.read_parquet('./data/train_title_fasttext.snappy.parquet')
test = pd.read_parquet('./data/test_title_fasttext.snappy.parquet')

In [0]:
from sklearn.decomposition import PCA

N_COMPONENTS = 32

pca = PCA(n_components=N_COMPONENTS)
train_pca = pd.DataFrame(pca.fit_transform(train.values), columns=['title_ft_%d'%i for i in range(N_COMPONENTS)])
test_pca = pd.DataFrame(pca.transform(test.values), columns=['title_ft_%d'%i for i in range(N_COMPONENTS)])

In [0]:
train_pca.to_parquet('./data/train_title_fasttext_32.snappy.parquet')
test_pca.to_parquet('./data/test_title_fasttext_32.snappy.parquet')

In [20]:
!gsutil cp ./data/train_title_fasttext_32.snappy.parquet gs://{os.environ['GCP_BUCKET']}/train_prep_fasttext_32.snappy.parquet
!gsutil cp ./data/test_title_fasttext_32.snappy.parquet gs://{os.environ['GCP_BUCKET']}/test_prep_fasttext_32.snappy.parquet


Copying file://./data/train_title_fasttext_32.snappy.parquet [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/198.5 MiB.                                    
Copying file://./data/test_title_fasttext_32.snappy.parquet [Content-Type=application/octet-stream]...
\
Operation completed over 1 obje

In [0]:
# TODO: very likely read_parquet has parquet file size limit, see below... pull request for pandas...
for t in types:
  df = pd.read_parquet('./data/%s_prep.snappy.parquet'%t)
  for col in str_cols:
    print('%s %s'%(t, col))
    ft = pd.read_parquet('./data/%s_%s_fasttext.snappy.parquet'%(t, col))
    print(1)
    df = pd.concat([df, ft], axis=1)
    print(2)

    del ft
    gc.collect()
  
  print(3)
  path = './data/%s_prep_fasttext.snappy.parquet'%t
  df.to_parquet(path)
  !gsutil cp {path} gs://{os.environ['GCP_BUCKET']}/{t}_prep_fasttext.snappy.parquet
  del df
  gc.collect()

train title
1
2
3
Copying file://./data/train_prep_fasttext.snappy.parquet [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][  2.2 GiB/  2.2 GiB]   78.8 MiB/s                                   
Operation completed over 1 objects/2.2 GiB.                                      
test title
1
2
3
Copying file://./data/test

In [0]:
!ls ./data

test_prep_fasttext.snappy.parquet   train_prep_fasttext.snappy.parquet
test_prep.snappy.parquet	    train_prep.snappy.parquet
test_title_fasttext.snappy.parquet  train_title_fasttext.snappy.parquet


In [0]:
df = pd.read_parquet('./data/train_prep_fasttext.snappy.parquet')

ArrowIOError: ignored

In [0]:
import pandas as pd
df = pd.read_parquet('./data/%s_prep.snappy.parquet'%('train'))
ft = pd.read_parquet('./data/%s_%s_fasttext.snappy.parquet'%('train', 'title')).fillna(0.)

df2 = pd.concat([df, ft], axis=1)

In [0]:
df2[df2.columns[len(df2.columns)//2:]].to_parquet('./data/huehue.snappy.parquet')

In [0]:
import pandas as pd
df3 = pd.read_parquet('./data/huehue.snappy.parquet')

In [0]:
df3

,title_ft_130,title_ft_131,title_ft_132,title_ft_133,title_ft_134,title_ft_135,title_ft_136,title_ft_137,title_ft_138,title_ft_139,...,title_ft_290,title_ft_291,title_ft_292,title_ft_293,title_ft_294,title_ft_295,title_ft_296,title_ft_297,title_ft_298,title_ft_299
0,-0.011269,0.012303,0.008749,0.012676,-0.014230,-0.027369,-0.022806,0.020298,0.047617,-0.037541,...,0.015485,0.011385,0.048488,-0.010008,0.049212,-0.011778,0.019683,0.056294,-0.045042,0.004157
1,-0.002584,-0.000724,-0.006417,-0.036129,-0.034260,-0.018286,-0.045239,0.042506,0.080846,0.007356,...,0.053226,-0.013267,0.064455,-0.008273,0.045398,0.019470,-0.000444,0.051863,-0.020145,0.015462
2,-0.042874,-0.038938,0.011918,-0.031646,0.039220,-0.023667,0.020220,0.033414,0.062992,0.004582,...,-0.006389,-0.009522,-0.009377,-0.020947,-0.018273,0.049952,-0.023931,0.015219,0.016215,0.058882
3,0.075212,-0.003082,0.005670,-0.002929,-0.133549,0.046292,-0.060073,0.026664,0.018802,-0.015053,...,-0.182703,-0.037711,-0.000575,-0.048422,0.065297,-0.040517,0.056970,0.056424,-0.024958,-0.050024
4,-0.094483,0.012982,0.038054,-0.010634,0.010813,-0.000225,0.030796,-0.008565,0.012823,0.002824,...,0.005630,0.017882,-0.082217,-0.004412,-0.006562,0.025846,-0.003078,0.028420,0.025882,0.019413
5,-0.020805,-0.047897,0.021995,0.023618,-0.043329,-0.015721,0.025757,-0.038530,0.041925,-0.006982,...,-0.112320,-0.006953,-0.035680,0.026139,0.032322,-0.016283,-0.058990,0.014655,-0.023890,-0.011254
6,-0.060789,-0.006341,-0.035080,0.038623,-0.022311,-0.054309,0.057145,0.009023,0.052482,-0.029995,...,0.011109,-0.010312,-0.042143,-0.020804,0.001975,0.030417,-0.008773,0.009048,-0.013167,-0.038139
7,-0.034237,-0.027148,0.000425,-0.073124,-0.054927,0.006394,-0.005113,0.044034,0.052061,-0.053266,...,0.042889,-0.062890,-0.027755,-0.026094,0.043779,0.028303,-0.004009,0.021261,0.052556,-0.021613
8,-0.032796,-0.060351,-0.006284,-0.066684,0.010678,-0.048039,0.002289,0.063892,0.050174,0.067772,...,0.011442,-0.115703,-0.050633,0.092213,0.058273,0.023918,-0.055356,0.020984,0.058672,0.021029
9,0.005844,-0.088073,-0.031882,-0.012425,-0.026267,-0.012555,-0.007100,0.051494,0.019585,-0.059485,...,-0.023484,-0.041751,-0.011793,-0.038451,0.081052,-0.028705,-0.038855,0.042457,0.009086,-0.024705
